1. Extracción de Datos a Nivel Nacional:
Extraer datos de generación de energía renovable y no renovable a nivel nacional desde el 1 de enero de 2011 hasta el 31 de diciembre de 2022. La información debe ser recopilada a nivel mensual.
Transformar los datos para incluir una columna nueva que identifique si la energía es renovable o no renovable.


In [1]:
import time
# Importar librería para trabajar con archivos JSON
# -----------------------------------------------------------------------
import json

# Importar librerías para manipulación y análisis de datos
# -----------------------------------------------------------------------
import pandas as pd
# importamos las librerías que necesitamos


# Uso de API's
# -----------------------------------------------------------------------
import requests

# Importar librerías para procesamiento de texto
# -----------------------------------------------------------------------
import re

# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# importamos las librerías que necesitaremos





In [2]:

def llamar_api(url):
    """
    Realiza una llamada a una API utilizando la URL proporcionada.

    Parameters:
    -----------
    url (str): La URL de la API que se va a llamar.

    Returns:
    --------
    dict or None: Un diccionario con los datos de respuesta de la API si la llamada fue exitosa (código de estado 200).
                  None si la llamada falló o no se pudo autenticar correctamente.
    """
    llamada = requests.get(url)  # Realiza una solicitud GET a la URL proporcionada y almacena la respuesta en 'llamada'.

    print(f"La llamada a la API nos ha dado una respuesta de tipo: {llamada.status_code}")  # Imprime el código de estado de la respuesta.

    if llamada.status_code != 200:  # Comprueba si la respuesta no fue exitosa (código de estado distinto de 200).
        print(f"El motivo por el que la llamada falló es {llamada.reason}")  # Imprime la razón de la falla.
    else:
        return llamada.json()  # Si la llamada fue exitosa, devuelve los datos de respuesta en formato JSON.


In [3]:
energia_obtenida_todos = []

for año in range(2011,2024):
    for mes in range(1,13):
        energia_obtenida = llamar_api(f"https://apidatos.ree.es/es/datos/generacion/estructura-generacion?start_date={año}-{mes}-01T00:00&end_date={año}-{mes}-31T23:59&time_trunc=month")["included"]
        energia_obtenida_todos.extend(energia_obtenida)



La llamada a la API nos ha dado una respuesta de tipo: 200
La llamada a la API nos ha dado una respuesta de tipo: 200
La llamada a la API nos ha dado una respuesta de tipo: 200
La llamada a la API nos ha dado una respuesta de tipo: 200
La llamada a la API nos ha dado una respuesta de tipo: 200
La llamada a la API nos ha dado una respuesta de tipo: 200
La llamada a la API nos ha dado una respuesta de tipo: 200
La llamada a la API nos ha dado una respuesta de tipo: 200
La llamada a la API nos ha dado una respuesta de tipo: 200
La llamada a la API nos ha dado una respuesta de tipo: 200
La llamada a la API nos ha dado una respuesta de tipo: 200
La llamada a la API nos ha dado una respuesta de tipo: 200
La llamada a la API nos ha dado una respuesta de tipo: 200
La llamada a la API nos ha dado una respuesta de tipo: 200
La llamada a la API nos ha dado una respuesta de tipo: 200
La llamada a la API nos ha dado una respuesta de tipo: 200
La llamada a la API nos ha dado una respuesta de tipo: 2

In [4]:
energia_limpio = {
"tipo_energia" : [],
"tipo_generacion" : [],
"fecha" : [],
"valor" : []
}

for i in energia_obtenida_todos:
    energia_limpio["tipo_energia"].append(i["type"])
    energia_limpio["tipo_generacion"].append(i["attributes"]["type"])
    energia_limpio["fecha"].append(i["attributes"]["values"][0]["datetime"])
    energia_limpio["valor"].append(i["attributes"]["values"][0]["value"])


In [5]:
energia_obtenida_df = pd.DataFrame(energia_limpio)

In [6]:
energia_obtenida_df

,tipo_energia,tipo_generacion,fecha,valor
0,Hidráulica,Renovable,2011-01-01T00:00:00.000+01:00,5.119512e+06
1,Turbinación bombeo,No-Renovable,2011-01-01T00:00:00.000+01:00,2.839449e+05
2,Nuclear,No-Renovable,2011-01-01T00:00:00.000+01:00,4.706501e+06
3,Carbón,No-Renovable,2011-01-01T00:00:00.000+01:00,2.303088e+06
4,Fuel + Gas,No-Renovable,2011-01-01T00:00:00.000+01:00,-7.511700e+02
...,...,...,...,...
2690,Otras renovables,Renovable,2023-12-01T00:00:00.000+01:00,2.545622e+05
2691,Cogeneración,No-Renovable,2023-12-01T00:00:00.000+01:00,1.184795e+06
2692,Residuos no renovables,No-Renovable,2023-12-01T00:00:00.000+01:00,1.118632e+05
2693,Residuos renovables,Renovable,2023-12-01T00:00:00.000+01:00,7.472553e+04


2. Extracción de Datos por Comunidad Autónoma:
Extraer datos de generación de energía renovable y no renovable por Comunidad Autónoma.
Utilizar el diccionario cod_comunidades para obtener los códigos de cada comunidad autónoma.
Transformar los datos para incluir una columna que especifique la comunidad autónoma y su identificador.

In [7]:
cod_comunidades = {'Ceuta': 8744,
                    'Melilla': 8745,
                    'Andalucía': 4,
                    'Aragón': 5,
                    'Cantabria': 6,
                    'Castilla - La Mancha': 7,
                    'Castilla y León': 8,
                    'Cataluña': 9,
                    'País Vasco': 10,
                    'Principado de Asturias': 11,
                    'Comunidad de Madrid': 13,
                    'Comunidad Foral de Navarra': 14,
                    'Comunitat Valenciana': 15,
                    'Extremadura': 16,
                    'Galicia': 17,
                    'Illes Balears': 8743,
                    'Canarias': 8742,
                    'Región de Murcia': 21,
                    'La Rioja': 20}

In [9]:
energia_comunidades_todas= []

for año in range(2011,2024):
    for mes in range(1,13):
        for comunidad in cod_comunidades.values():
            energia_obtenida_comunidad = llamar_api(f"https://apidatos.ree.es/es/datos/generacion/estructura-generacion?start_date={año}-{mes}-01T00:00&end_date={año}-{mes}-31T23:59&time_trunc=year&geo_trunc=electric_system&geo_limit=ccaa&geo_ids={comunidad}")["included"]
            print(f"He descargado la infromación para el año {año}, mes {mes} y ccaa {comunidad}.")
            energia_comunidades_todas.extend(energia_obtenida_comunidad)
            time.sleep(0.1*mes)
    with open("energia.json", "w") as fichero:
        json.dump(energia_comunidades_todas, fichero)
                



La llamada a la API nos ha dado una respuesta de tipo: 200
He descargado la infromación para el año 2011, mes 1 y ccaa 8744.
La llamada a la API nos ha dado una respuesta de tipo: 200
He descargado la infromación para el año 2011, mes 1 y ccaa 8745.
La llamada a la API nos ha dado una respuesta de tipo: 200
He descargado la infromación para el año 2011, mes 1 y ccaa 4.
La llamada a la API nos ha dado una respuesta de tipo: 200
He descargado la infromación para el año 2011, mes 1 y ccaa 5.
La llamada a la API nos ha dado una respuesta de tipo: 200
He descargado la infromación para el año 2011, mes 1 y ccaa 6.
La llamada a la API nos ha dado una respuesta de tipo: 200
He descargado la infromación para el año 2011, mes 1 y ccaa 7.
La llamada a la API nos ha dado una respuesta de tipo: 200
He descargado la infromación para el año 2011, mes 1 y ccaa 8.
La llamada a la API nos ha dado una respuesta de tipo: 200
He descargado la infromación para el año 2011, mes 1 y ccaa 9.
La llamada a la AP

In [ ]:
energia_limpio_comunidades = {
"tipo_energia" : [],
"tipo_generacion" : [],
"fecha" : [],
"valor" : []
}

for i in energia_obtenida_todos:
    energia_limpio["tipo_energia"].append(i["type"])
    energia_limpio["tipo_generacion"].append(i["attributes"]["type"])
    energia_limpio["fecha"].append(i["attributes"]["values"][0]["datetime"])
    energia_limpio["valor"].append(i["attributes"]["values"][0]["value"])

In [11]:
with open ("energia.json", "r") as fichero:


La llamada a la API nos ha dado una respuesta de tipo: 200


In [12]:
energia_obtenida_comunidad

[{'type': 'Motores diésel',
  'id': '10344',
  'groupId': '1',
  'attributes': {'title': 'Motores diésel',
   'description': None,
   'color': '#8DA69F',
   'type': 'No-Renovable',
   'magnitude': None,
   'composite': False,
   'last-update': '2018-12-21T10:43:15.000+01:00',
   'values': [{'value': 31944.179,
     'percentage': 0.9981782349367924,
     'datetime': '2011-01-01T00:00:00.000+01:00'}]}},
 {'type': 'Turbina de gas',
  'id': '1450',
  'groupId': '1',
  'attributes': {'title': 'Turbina de gas',
   'description': None,
   'color': '#00ccff',
   'type': 'No-Renovable',
   'magnitude': None,
   'composite': False,
   'last-update': '2018-12-21T10:43:18.000+01:00',
   'values': [{'value': 58.301,
     'percentage': 0.0018217650632076014,
     'datetime': '2011-01-01T00:00:00.000+01:00'}]}},
 {'type': 'Generación total',
  'id': '1',
  'groupId': '1',
  'attributes': {'title': 'Generación total',
   'description': None,
   'color': '#2b2e34',
   'type': 'Generación total',
   'ma